In [4]:
import os
import sys
import torch
import importlib

from peft import LoraConfig
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

In [5]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load environment variables
load_dotenv()

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [6]:
import src.train
import src.model
import data.sudoku

importlib.reload(src.train)
importlib.reload(src.model)
importlib.reload(data.sudoku)

from src.train import sft_train_lora
from src.model import identify_target_modules
from data.sudoku import Sudoku

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

dataset = Sudoku(data_file=os.environ['SUDOKU_PATH'], tokenizer=tokenizer)

dataset = Dataset.from_dict({"input_text" : [example["input_text"] for example in dataset]})

In [9]:
dataset[0]

{'input_text': 'Given the Sudoku puzzle 1..5.37..6.3..8.9......98...1.......8761..........6...........7.8.9.76.47...6.312, which has 27 clues and a difficulty rating of 2.2. Please solve for the final arrangement. ### Answer: 198543726643278591527619843914735268876192435235486179462351987381927654759864312'}

In [10]:
len(dataset)

3000000

In [10]:
target_modules = identify_target_modules(model, name_segment='self_attn')
print(target_modules)

['model.decoder.layers.0.self_attn.k_proj', 'model.decoder.layers.0.self_attn.v_proj', 'model.decoder.layers.0.self_attn.q_proj', 'model.decoder.layers.0.self_attn.out_proj', 'model.decoder.layers.1.self_attn.k_proj', 'model.decoder.layers.1.self_attn.v_proj', 'model.decoder.layers.1.self_attn.q_proj', 'model.decoder.layers.1.self_attn.out_proj', 'model.decoder.layers.2.self_attn.k_proj', 'model.decoder.layers.2.self_attn.v_proj', 'model.decoder.layers.2.self_attn.q_proj', 'model.decoder.layers.2.self_attn.out_proj', 'model.decoder.layers.3.self_attn.k_proj', 'model.decoder.layers.3.self_attn.v_proj', 'model.decoder.layers.3.self_attn.q_proj', 'model.decoder.layers.3.self_attn.out_proj', 'model.decoder.layers.4.self_attn.k_proj', 'model.decoder.layers.4.self_attn.v_proj', 'model.decoder.layers.4.self_attn.q_proj', 'model.decoder.layers.4.self_attn.out_proj', 'model.decoder.layers.5.self_attn.k_proj', 'model.decoder.layers.5.self_attn.v_proj', 'model.decoder.layers.5.self_attn.q_proj', 

In [11]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

In [12]:
sft_train_lora(
    base_model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=AutoTokenizer.from_pretrained("facebook/opt-350m"),
    adapter_name="sft_lora",
    response_template=" ### Answer:",
    lora_config=lora_config,
)

Map:   0%|          | 0/3000000 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/3000000 [00:00<?, ? examples/s]

/opt/anaconda3/envs/reasoning-decomp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/1125000 [00:00<?, ?it/s]

KeyboardInterrupt: 